<h4>1. 이미지와 객체 번호 저장</h4>

- 이미지와 해당 객체 번호를 JSON 파일에 저장함


In [10]:
import os
import cv2
import pickle
import shutil
import numpy as np 
from sklearn.neighbors import NearestNeighbors

# 이미지 저장 경로 설정
image_dir = "images"
os.makedirs(image_dir, exist_ok=True)

# 텍스트 파일 경로 설정
metadata_file = "metadata.txt"

# ORB 특징 추출 함수
def extract_features(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    orb = cv2.ORB_create()
    keypoints, descriptors = orb.detectAndCompute(image, None)
    return descriptors

# 이미지 추가 함수
def add_image(image_path, object_type, feature_file="features.pkl"):
    # 텍스트 파일 로드 또는 초기화
    if os.path.exists(metadata_file):
        with open(metadata_file, 'r') as f:
            lines = f.readlines()
    else:
        lines = []

    if os.path.exists(feature_file):
        with open(feature_file, 'rb') as f:
            existing_features = pickle.load(f)
    else:
        existing_features = np.array([]).reshape(0, 32)  # ORB 특징 벡터 크기

    new_features = extract_features(image_path)
    if new_features is None: 
        print("No features found in the image.")
        return False

    # 새로운 이미지 번호 생성
    new_id = len(lines) + 1

    existing_features = np.vstack([existing_features, new_features])
    
    # 텍스트 파일에 새로운 이미지 정보 추가
    with open(metadata_file, 'a') as f:
        f.write(f"{new_id},{image_path},{object_type}\n")
    
    with open(feature_file, 'wb') as f:
        pickle.dump(existing_features, f)
    
    print(f"Image {new_id} added with object type {object_type}.")
    return True 


<h4>TXT 파일 읽고 0,1,2,3 객체의 이미지 파일 가져오기</h4> 

In [67]:
import os

# 텍스트 파일 경로 설정
txt_file_path = "metadata.txt"
image_dir = "images"

# 함수 정의: 특정 객체 번호에 해당하는 이미지 가져오기
def get_images_by_object_types(txt_file, object_types, image_dir):
    images = []
    with open(txt_file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            parts = line.strip().split(',')
            if len(parts) == 3:
                image_path = parts[1].strip()
                obj_type = int(parts[2].strip())
                if obj_type in object_types and os.path.exists(image_path):
                    images.append(image_path)
    return images

# 특정 객체 번호 리스트 설정
object_types_to_search = [0, 1, 2, 3]

# 함수 호출
found_images = get_images_by_object_types(txt_file_path, object_types_to_search, image_dir)

# 결과 출력
if found_images:
    print("Found images:")
    for img_path in found_images:
        print(img_path)
else:
    print("No images found for the specified object types.")


FileNotFoundError: [Errno 2] No such file or directory: 'metadata.txt'

<h4>2. 특정 객체 번호에 해당하는 이미지 검색</h4>

- 메타데이터 파일에서 객체 번호 검색하여 해당 이미지를 반환하는함수

In [2]:
# 특정 객체 번호에 해당하는 이미지 검색 함수
def search_image_by_object_type(object_type):
    if not os.path.exists(metadata_file):
        print("No metadata found. Add images first.")
        return None
    
    with open(metadata_file, 'r') as f:
        metadata = json.load(f)
    
    matching_images = [item["path"] for item in metadata if item["object_type"] == object_type]
    
    if matching_images:
        return matching_images
    else:
        print(f"No images found with object type {object_type}.")
        return None

# 이미지 추가 예시
add_image('path_to_image_of_man.jpg', 0)
add_image('path_to_image_of_woman.jpg', 1)
add_image('path_to_image_of_bicycle.jpg', 2)
add_image('path_to_image_of_motorcycle.jpg', 3)

# 특정 객체 번호에 해당하는 이미지 검색 예시
object_type_to_search = 0  # 남자 객체 번호
result_images = search_image_by_object_type(object_type_to_search)
if result_images:
    for img_path in result_images:
        print(f"Image with object type {object_type_to_search}: {img_path}")


No features found in the image.
No features found in the image.
No features found in the image.
No features found in the image.
No metadata found. Add images first.


<h4>3. 필요한 라이브러리 및 경로 설정</h4>

In [3]:
import os
import json
import shutil

# JSON 파일 경로 설정
metadata_file = "metadata.json"

# 특정 객체 번호에 해당하는 이미지 복사 경로 설정
target_dir = "filtered_images"
os.makedirs(target_dir, exist_ok=True)


<h4>4. 특정 객체 번호에 해당하는 이미지 복사 함수</h4>


In [4]:
def copy_images_by_object_type(object_type, target_dir):
    if not os.path.exists(metadata_file):
        print("No metadata found. Add images first.")
        return
    
    with open(metadata_file, 'r') as f:
        metadata = json.load(f)
    
    matching_images = [item["path"] for item in metadata if item["object_type"] == object_type]
    
    if not matching_images:
        print(f"No images found with object type {object_type}.")
        return
    
    for img_path in matching_images:
        if os.path.exists(img_path):
            shutil.copy(img_path, target_dir)
            print(f"Copied {img_path} to {target_dir}.")
        else: 
            print(f"Image {img_path} not found.")


<h4>5. txt 파일 내에서 지정한 객체 번호 이외의 숫자 제거</h4>

In [5]:
def filter_txt_file(txt_file_path, valid_object_types):
    if not os.path.exists(txt_file_path):
        print(f"{txt_file_path} not found.")
        return
    
    with open(txt_file_path, 'r') as f:
        lines = f.readlines()
    
    filtered_lines = [line for line in lines if int(line.strip()) in valid_object_types]
    
    with open(txt_file_path, 'w') as f:
        f.writelines(filtered_lines)
    
    print(f"Filtered lines in {txt_file_path}.")

# 특정 객체 번호 예시
object_types_to_keep = [0, 2]  # 예: 남자와 자전거만 유지

# 이미지 복사 예시
for object_type in object_types_to_keep:
    copy_images_by_object_type(object_type, target_dir)

# txt 파일 필터링 예시
filter_txt_file('path_to_your_file.txt', object_types_to_keep)


No metadata found. Add images first.
No metadata found. Add images first.
path_to_your_file.txt not found.


<h4> 6. 이미지 데이터베이스 준비</h4>

In [6]:
import os

# 이미지 저장 경로 설정
image_dir = "images"
os.makedirs(image_dir, exist_ok=True)


<h4> 7. 이미지 특징 추출</h4>

In [7]:
import cv2
import numpy as np

def extract_features(image_path):
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        orb = cv2.ORB_create()
        keypoints, descriptors = orb.detectAndCompute(image, None)
        return descriptors

<h4>8. 중복 이미지 탐지</h4>

In [8]:
from sklearn.neighbors import NearestNeighbors

def is_duplicate(image_path, existing_features, threshold=0.7):
    new_features = extract_features(image_path)
    if new_features is None:
        return False
    
    # NearestNeighbors 모델 생성 및 학습
    nn = NearestNeighbors(n_neighbors=1, metric='hamming')
    nn.fit(existing_features)
    
    distances, indices = nn.kneighbors(new_features)
    avg_distance = np.mean(distances)
    
    return avg_distance < threshold


<h4>9. 새로운 이미지 추가 시 학습 데이터 업데이트</h4>

In [9]:
import pickle

def add_image(image_path, feature_file="features.pkl"):
    if os.path.exists(feature_file):
        with open(feature_file, 'rb') as f:
            existing_features = pickle.load(f)
    else:
        existing_features = np.array([]).reshape(0, 32)  # ORB 특징 벡터 크기

    if is_duplicate(image_path, existing_features):
        print(f"{image_path} is a duplicate.")
        return False
    else:
        new_features = extract_features(image_path)
        existing_features = np.vstack([existing_features, new_features])
        with open(feature_file, 'wb') as f:
            pickle.dump(existing_features, f)
        print(f"{image_path} added.")
        return True


<h4>10. 검색을 통해 이미지 추출</h4>

In [76]:
def search_image(query_image_path, feature_file="features.pkl"):
    if not os.path.exists(feature_file):
        print("No features found. Add images first.")
        return None
    
    with open(feature_file, 'rb') as f:
        existing_features = pickle.load(f)
    
    query_features = extract_features(query_image_path)
    if query_features is None:
        print("No features found in query image.")
        return None
    
    nn = NearestNeighbors(n_neighbors=1, metric='hamming')
    nn.fit(existing_features)
    
    distances, indices = nn.kneighbors(query_features)
    closest_index = indices[0][0]
    
    return existing_features[closest_index]

# 이미지 추가 예시
add_image('C:\\Users\\kiot\\Yolo V5\\yolov5\\DatasetProgram\\images\\images/214501174_6db1f4d69c.jpg')

# 이미지 검색 예시
result = search_image('214501174_6db1f4d69c.jpg')
if result is not None:
    print("Matching image found.") 


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 32 and the array at index 1 has size 1